---

## Исследование надёжности заёмщиков — анализ банковских данных

---

<u>**Цель проекта**</u>: исследование факторов, влияющих на на факт погашения кредита в срок.

<u>**Сферы деятельности компаний**</u>: банковская сфера, кредитование.

<u>**Навыки и инструменты**</u>: Pandas, PyMystem3, Python, лемматизация, предобработка данных


### 1. Получение общей информации 

In [1]:
import pandas as pd
import numpy as np
import pymystem3
from pymystem3 import Mystem

Получим общую информацию о данных в файле:

In [2]:
data = pd.read_csv(...)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Сразу обращают на себя внимание **пропуски** в стоблцах *'days_employed'* и *'total_income'*. Очевидно, природа появления пропусков в этих двух столбцах одинакова.

In [4]:
print('Доля строк, имеющих пропуски данных: {:.0%}'
      .format(data.isna()['days_employed'].sum()
              / data['children'].count()))

Доля строк, имеющих пропуски данных: 10%


Предстоит заполнение пропусков: 10% — существенная доля.

Получим первые 10 строк таблицы:

In [5]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим минимальные и максимальные значения количественных переменных:

In [6]:
print('Минимальное количество детей: {:.0f};'.format(data['children'].min()))
print('Максимальное количество детей: {:.0f};'.format(data['children'].max()))

Минимальное количество детей: -1;
Максимальное количество детей: 20;


Некоторые клиенты банка имеют **отрицательное количество детей**. Возможно, данная аномалия была вызвана опечатками при вводе данных.
20 детей — это сильно, но (при определённых обстоятельствах :D) возможно, поэтому оставим строки с этим значением в выборке.

Объявим для удобства две функции автоматической адаптации слов "*день*" и "*год*" к стоящим перед ними числовым значениям:

In [7]:
def days_number(days):
    days_ending = ['день', 'дня', 'дней']
    if days % 10 == 1 and days % 100 != 11:
        p = 0
    elif (2 <= days%10 <= 4) and (days%100 < 10 or days%100 >= 20):
        p = 1
    else:
        p = 2
    return str(days)+' '+days_ending[p]

def years_number(years):
    years_ending = ['год', 'года', 'лет']
    if years%10 == 1 and years%100 != 11:
        p = 0
    elif years%10 <= 4 and years%10 != 0 and (years%100 < 10 or years%100 >=20):
        p = 1
    else:
        p = 2
    return str(years)+' '+years_ending[p]

In [8]:
print('Минимальный рабочий стаж: ', days_number(data['days_employed'].min()), ';', sep='')
print('Максимальный рабочий стаж: ', days_number(data['days_employed'].max()), ';', sep='')

Минимальный рабочий стаж: -18388.949900568383 дней;
Максимальный рабочий стаж: 401755.40047533 дней;


"Как хорошо ничего не делать, а потом отдохнуть". Некоторые клиенты имеют **отрицательный стаж**, что, несомненно, является аномалией. Причина появления данной аномалии может быть связана с ошибкой в формуле, вычисляющей общий рабочий стаж путём вычитания даты и времени начала работы из даты и времени окончания.

In [9]:
data_years_employed_max = int(round(data['days_employed'].max()/365))
print('Максимальный рабочий стаж: ', years_number(data_years_employed_max), ';', sep='')

Максимальный рабочий стаж: 1101 год;


Ещё одна аномалия. Только человек, сумевший обрести бессмертие, мог бы отдать работе **1101 год** своей жизни. Целесообразно в выборку не включать клиентов, чей стаж превышает разность максимального значения возраста клиентов в выборке и 18, то есть 57  лет.

In [10]:
print('Минимальный возраст: ',  years_number(data['dob_years'].min()), ';', sep='')
print('Максимальный возраст: ', years_number(data['dob_years'].max()), ';', sep='')

Минимальный возраст: 0 лет;
Максимальный возраст: 75 лет;


Из роддома — на работу! Поскольку для того, чтобы взять кредит, лицо должно быть дееспособным, необходимо исключить из выборки клиентов, **не достигших совершеннолетия**.

In [11]:
print('Минимальный доход: {:.0f} руб.;'.format(data['total_income'].min()))
print('Максимальный доход: {:.0f} руб.;'.format(data['total_income'].max()))

Минимальный доход: 20667 руб.;
Максимальный доход: 2265604 руб.;


С доходами обошлось без аномалий.

In [12]:
print('Аномалии:')
print('Строки с отрицительными значениями количества детей:',
      data[data['children'] == -1]['children'].count())
print()
print('Строки со значениями возраста клиентов, меньшими 18 лет:',
      data[data['dob_years'] < 18]['dob_years'].count())
print('Из них:')
print('Строки с нулевыми значениями возраста клиентов:',
      data[data['dob_years'] == 0]['dob_years'].count())

Аномалии:
Строки с отрицительными значениями количества детей: 47

Строки со значениями возраста клиентов, меньшими 18 лет: 101
Из них:
Строки с нулевыми значениями возраста клиентов: 101


### Вывод

Данных много. Со стандартными именами стоблцов можно работать.

В процессе изучения данных были выявлены следующие аномалии:

- **Некоторые клиенты банка имеют отрицательное количество детей;**
- **Имеются люди с отрицательным рабочим стажем;**
- **Также имеются и люди, чей стаж заметно больше 80 лет;**
- **Присутствуют несовершеннолетние.**

Предстоит привести к адекватному виду значения в столбцах *'education'* и *'purpose'*, заменить отрицательные значения количества детей и дней рабочего стажа их модулями, а также исключить из выборки несовершеннолетних клиентов и людей, чей общий рабочий стаж превышает 80 лет.

### 2. Предобработка данных

#### Обработка пропусков

Вычислим **общее количество пропусков** в каждом стоблце:

In [13]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заменим значения столбца *'days_employed'* на их модули, чтобы устранить допущенную при сборе данных расчётную ошибку:

In [14]:
data['days_employed'] = data['days_employed'].apply(abs)

Необходимо оценить, каков процент клиентов, чей рабочий стаж, согласно предоставленным данным, превышает 57 лет:

In [15]:
max_days_employed = (75 - 18) * 365
print('Доля клиентов с рабочим стажем выше 57 лет: {:.2%}'
      .format(data[data['days_employed'] > max_days_employed]['days_employed'].count()
              /data['days_employed'].count()))

Доля клиентов с рабочим стажем выше 57 лет: 17.80%


Найдём среднее значение рабочего стажа в выборке с учётом дополнительного условия, сделаем его целым, чтобы было проще работать со значениями:

In [16]:
days_employed_mean = int(data[data['days_employed'] <= max_days_employed]['days_employed']
                         .mean())

Найдём медиану выборки с учётом дополнительного условия, также сделаем число целым:

In [17]:
days_employed_median = int(data[data['days_employed'] <= max_days_employed]['days_employed']
                           .median())

Заменим неудовлетворяющие дополнительному условию значения медианными:

In [18]:
for i in data:
    data.loc[data['days_employed'] > max_days_employed, 'days_employed'] = days_employed_median

Убедимся, что теперь максимальное значение удовлетворяет дополнительному условию:

In [19]:
data_years_employed_max = int(round(data['days_employed'].max()/365))
print('Максимальный рабочий стаж: ', years_number(data_years_employed_max), ';', sep='')

Максимальный рабочий стаж: 50 лет;


Выведем на экран **минимум**, **максимум**, **математическое ожидание** и **медиану**:

In [20]:
days_employed_min = int(data['days_employed'].min())
days_employed_max = int(data['days_employed'].max())

print('Минимальный рабочий стаж: ', days_number(days_employed_min), ';', sep='')
print('Максимальный рабочий стаж: ', days_number(days_employed_max), ';', sep='')
print('Средний рабочий стаж: ', days_number(days_employed_mean), ';', sep='')
print('Медианное значение рабочего стажа: ', days_number(days_employed_median), ';', sep='')

Минимальный рабочий стаж: 24 дня;
Максимальный рабочий стаж: 18388 дней;
Средний рабочий стаж: 2353 дня;
Медианное значение рабочего стажа: 1630 дней;


Аналогичные операции проведём и со столбцом *'total_income'*:

In [21]:
total_income_mean = int(data['total_income'].mean())
total_income_median = int(data['total_income'].median())
print('Минимальный общий доход: {:.0f}'.format(data['total_income'].min()), 'руб.')
print('Максимальный общий доход: {:.0f}'.format(data['total_income'].max()), 'руб.')
print('Среднее значение общего дохода:', total_income_mean, 'руб.')
print('Медианное значение общего дохода:', total_income_median, 'руб.')

Минимальный общий доход: 20667 руб.
Максимальный общий доход: 2265604 руб.
Среднее значение общего дохода: 167422 руб.
Медианное значение общего дохода: 145017 руб.


**Заполним пропуски характерными для данной выборки значениями**. С учётом большой разницы между минимальным и максимальным значениями, в качестве характерных целесообразно использовать медианные значения выборки:

In [22]:
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)

Целые рабочие дни смотрятся приятнее:

In [23]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Выведем на экран первые 10 строк получившейся таблицы:

In [24]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Разберёмся со столбцами *'children'* и *'dob_years'*.
Для начала **заменим отрицательные значения** в столбце *'children'* их модулями и убедимся, что отрицательных значений не осталось:

In [25]:
data['children'] = data['children'].apply(abs)
print('Минимальное количество детей: {:.0f};'.format(data['children'].min()))
print()

Минимальное количество детей: 0;



Разберёмся с **возрастом** клиентов:

In [26]:
print('Доля строк со значениями возраста, не удовлетворяющими условию: {:.2%}'
      .format(data[data['dob_years'] == 0]['dob_years'].count() / data['dob_years'].count()))


Доля строк со значениями возраста, не удовлетворяющими условию: 0.47%


Очевидно, влияние таких строк на общую выборку минимально. Оставим в выборке только строки, значение возраста в которых не менее 18 лет. Убедимся, что минимальный возраст клиентов после удаления строк действительно соответствует нашим условиям:

In [27]:
data = data[data['dob_years'] >= 18]
print('Минимальный возраст клиентов: ', years_number(data['dob_years'].min()), '.', sep='')

Минимальный возраст клиентов: 19 лет.


Вычислим **математическое ожидание** и **медиану** для столбца *'dob_years'*:

In [28]:
dob_years_mean = int(data[data['dob_years'] != 0]['dob_years'].mean())
dob_years_median = int(data[data['dob_years'] != 0]['dob_years'].median())
print('Среднее значение возраста: ',  years_number(dob_years_mean), ';', sep='')
print('Медианное значение возраста: ',  years_number(dob_years_median), '.', sep='')

Среднее значение возраста: 43 года;
Медианное значение возраста: 43 года.


Просмотрим первые 25 строк таблицы:

In [29]:
data.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Пропуски присутствуют в столбцах *'days_employed'* и *'total_income'*. В рамках поставленной задачи присутствующие в строках с пропусками данные представляют ценность. В связи с этим, было принято решение заменить пропуски характерными значениями. В виду большого разброса, медианные значения предпочтительнее для использования в качестве характерных.

### Замена типа данных

Сделаем все заглавные буквы ячеек в столбцах *'education'*, *'family_status'*, *'income_type'* и *'purpose'* **строчными** и просмотрим первые 10 строк таблицы, чтобы убедиться, что всё идёт по плану:

In [30]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Просмотрим уникальные значения в столбце *'education'*:

In [31]:
print(data['education'].value_counts())

среднее                15169
высшее                  5225
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64


Люди с образованием выше среднего реже приходят в банк, чтобы взять кредит.

Просмотрим уникальные значения в столбце *'family_status'*:

In [32]:
print(data['family_status'].value_counts())

женат / замужем          12331
гражданский брак          4156
не женат / не замужем     2797
в разводе                 1185
вдовец / вдова             955
Name: family_status, dtype: int64


Чаще всего берут кредит те, у кого есть вторая половинка.

Просмотрим уникальные значения в столбце *'gender'*:

In [33]:
print(data['gender'].value_counts())

F      14164
M       7259
XNA        1
Name: gender, dtype: int64


Женщинам кредит нужнее, чем мужчинам.

Во время просмотра значений замечаем **аномалию**: пол "XNA".

Впрочем, в рамках поставленной задачи зависимость чего-либо от значений столбца *'gender'* нас не интересуют. Поскольку аномальное значение всего одно, можно в дальнейшем исключить эту строку из выборки:

In [34]:
data = data[data['gender'] != 'XNA']
print(data['gender'].value_counts())

F    14164
M     7259
Name: gender, dtype: int64


Просмотрим уникальные значения в столбце *'income_type'*:

In [35]:
print(data['income_type'].value_counts())

сотрудник          11064
компаньон           5064
пенсионер           3836
госслужащий         1453
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


### Вывод

Все заглавные буквы, содержащиеся в столбцах *'education'*, *'family_status'*, *'income_type'* и *'purpose'*, сделаны **строчными**. С такими данными намного удобнее работать. Просмотрены уникальные значения, содержащиеся в столбцах со значениями типа "string". Чаще всего берут кредит люди, не имеющие высшего образования. Женщин среди клиентов вдвое больше мужчин.

### Обработка дубликатов

Посчитаем количество дубликатов:

In [36]:
print('Продублировано строк: ', data.duplicated().sum(), '.', sep='')
print('Доля дубликатов: {:.2%}'.format(data.duplicated().sum() / data['children'].count()))

Продублировано строк: 71.
Доля дубликатов: 0.33%


Удалим дубликаты применением drop_duplicates() с последующим восстановлением правильной нумерации:

In [37]:
data = data.drop_duplicates().reset_index(drop=True)

### Вывод

Незначительную долю строк составляли дубликаты. Возможная причина — ошибки ввода. Дубликаты были исключены методом *drop_duplicates()*.

### Лемматизация

Для лемматизации используем pymystem3:

In [38]:
m = Mystem()
purposes = ['жилье', 'образование', 'свадьба', 'автомобиль', 'недвижимость']
def lemmatize_purposes(purpose):
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        if lemma in purposes:
            return lemma

Среди целей кредитов можно выделить 5 групп, содержащих соответственно слова "жильё", "образование", "свадьба", "автомобиль" и "недвижимость". Посчитаем количество строк, содержащих нужные нам леммы:

In [39]:
data['purposes_lemmatized'] = data['purpose'].apply(lemmatize_purposes)
data['purposes_lemmatized'].value_counts()

недвижимость    6327
жилье           4436
автомобиль      4284
образование     3995
свадьба         2310
Name: purposes_lemmatized, dtype: int64

### Вывод

Наиболее частая цель кредита — жильё и недвижимость.

### Категоризация данных

Создадим **набор словарей**, позволяющих дать ответы на поставленные в кейсе вопросы.

1. "Есть ли зависимость между наличием детей и возвратом кредита в срок?"

Добавим в словарь столбец, характеризующий наличие или отсутствие детей у клиента:

In [40]:
data_children_dict = data[['children', 'debt']]

def children_group(children_count):
    if children_count == 0:
        return 'нет детей'
    return 'есть дети'

data_children_dict['children_groups'] = data_children_dict['children'].apply(children_group)

C:\Users\Boris\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
children_0_debt_0 = (data_children_dict
                     [(data_children_dict['children_groups'] == 'нет детей')
                      & (data_children_dict['debt'] == 0)]['debt'].count())
children_0_debt_1 = (data_children_dict[(data_children_dict['children_groups'] == 'нет детей')
                                        &(data_children_dict['debt'] == 1)]['debt'].count())
children_1_debt_0 = (data_children_dict[(data_children_dict['children_groups'] == 'есть дети')
                                        &(data_children_dict['debt'] == 0)]['debt'].count())
children_1_debt_1 = (data_children_dict[(data_children_dict['children_groups'] == 'есть дети')
                                        &(data_children_dict['debt'] == 1)]['debt'].count())

2. "Есть ли зависимость между семейным положением и возвратом кредита в срок?"

In [42]:
data_family_status_dict = data[['family_status', 'debt']]

In [43]:
single_debt_0 = (data_family_status_dict
                 [(data_family_status_dict['family_status'] == 'не женат / не замужем')
                  &(data_family_status_dict['debt'] == 0)]['family_status'].count())
single_debt_1 = (data_family_status_dict
                 [(data_family_status_dict['family_status'] == 'не женат / не замужем')
                  &(data_family_status_dict['debt'] == 1)]['family_status'].count())

civil_marriage_debt_0 = (data_family_status_dict
                         [(data_family_status_dict['family_status'] == 'гражданский брак')
                          &(data_family_status_dict['debt'] == 0)]['family_status'].count())
civil_marriage_debt_1 = (data_family_status_dict
                         [(data_family_status_dict['family_status'] == 'гражданский брак')
                          &(data_family_status_dict['debt'] == 1)]['family_status'].count())

married_debt_0 = (data_family_status_dict
                  [(data_family_status_dict['family_status'] == 'женат / замужем')
                   &(data_family_status_dict['debt'] == 0)]['family_status'].count())
married_debt_1 = (data_family_status_dict
                  [(data_family_status_dict['family_status'] == 'женат / замужем')
                   &(data_family_status_dict['debt'] == 1)]['family_status'].count())

divorced_debt_0 = (data_family_status_dict
                   [(data_family_status_dict['family_status'] == 'в разводе')
                    &(data_family_status_dict['debt'] == 0)]['family_status'].count())
divorced_debt_1 = (data_family_status_dict
                   [(data_family_status_dict['family_status'] == 'в разводе')
                    &(data_family_status_dict['debt'] == 1)]['family_status'].count())

widower_debt_0 = (data_family_status_dict
                  [(data_family_status_dict['family_status'] == 'вдовец / вдова')
                   &(data_family_status_dict['debt'] == 0)]['family_status'].count())
widower_debt_1 = (data_family_status_dict
                  [(data_family_status_dict['family_status'] == 'вдовец / вдова')
                   &(data_family_status_dict['debt'] == 1)]['family_status'].count())

3. "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

Вычислим 20-й, 40-й, 60-й, 80-й и 100-й процентили, разделив выборку на 5 групп, отражающих соответственно уровни дохода: "низкий", "ниже среднего", "средний", "выше среднего" и "высокий". Добавим новый столбец к словарю:

In [44]:
data_total_income_dict = data[['total_income', 'debt']]

total_income_percentile_20 = int(np.percentile(data_total_income_dict['total_income'], 20))
total_income_percentile_40 = int(np.percentile(data_total_income_dict['total_income'], 40))
total_income_percentile_60 = int(np.percentile(data_total_income_dict['total_income'], 60))
total_income_percentile_80 = int(np.percentile(data_total_income_dict['total_income'], 70))
total_income_percentile_100 = int(np.percentile(data_total_income_dict['total_income'], 100))

def income_level(income):
    if income <= total_income_percentile_20:
        return 'низкий'
    if income > total_income_percentile_20 and income <= total_income_percentile_40:
        return 'ниже среднего'
    if income > total_income_percentile_40 and income <= total_income_percentile_60:
        return 'средний'
    if income > total_income_percentile_60 and income <= total_income_percentile_80:
        return 'выше среднего'
    if income > total_income_percentile_80 and income <= total_income_percentile_100:
        return 'высокий'
    
data_total_income_dict['income_level'] = data_total_income_dict['total_income'].apply(income_level)

C:\Users\Boris\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
percentile_20_debt_0 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'низкий')
                         &(data_total_income_dict['debt'] == 0)]['total_income'].count())
percentile_20_debt_1 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'низкий')
                         &(data_total_income_dict['debt'] == 1)]['total_income'].count())

percentile_40_debt_0 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'ниже среднего')
                         &(data_total_income_dict['debt'] == 0)]['total_income'].count())
percentile_40_debt_1 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'ниже среднего')
                         &(data_total_income_dict['debt'] == 1)]['total_income'].count())

percentile_60_debt_0 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'средний')
                         &(data_total_income_dict['debt'] == 0)]['total_income'].count())
percentile_60_debt_1 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'средний')
                         &(data_total_income_dict['debt'] == 1)]['total_income'].count())

percentile_80_debt_0 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'выше среднего')
                         &(data_total_income_dict['debt'] == 0)]['total_income'].count())
percentile_80_debt_1 = (data_total_income_dict
                        [(data_total_income_dict['income_level'] == 'выше среднего')
                         &(data_total_income_dict['debt'] == 1)]['total_income'].count())

percentile_100_debt_0 = (data_total_income_dict
                         [(data_total_income_dict['income_level'] == 'высокий')
                          &(data_total_income_dict['debt'] == 0)]['total_income'].count())
percentile_100_debt_1 = (data_total_income_dict
                         [(data_total_income_dict['income_level'] == 'высокий')
                          &(data_total_income_dict['debt'] == 1)]['total_income'].count())

4. "Как разные цели кредита влияют на его возврат в срок?"

In [46]:
data_purpose_dict = data[['purposes_lemmatized', 'debt']]

In [47]:
car_debt_0 = (data_purpose_dict
              [(data_purpose_dict['purposes_lemmatized'] == 'автомобиль')
               &(data_purpose_dict['debt'] == 0)]['purposes_lemmatized'].count())
car_debt_1 = (data_purpose_dict
              [(data_purpose_dict['purposes_lemmatized'] == 'автомобиль')
               &(data_purpose_dict['debt'] == 1)]['purposes_lemmatized'].count())

education_debt_0 = (data_purpose_dict
                    [(data_purpose_dict['purposes_lemmatized'] == 'образование')
                     &(data_purpose_dict['debt'] == 0)]['purposes_lemmatized'].count())
education_debt_1 = (data_purpose_dict
                    [(data_purpose_dict['purposes_lemmatized'] == 'образование')
                     &(data_purpose_dict['debt'] == 1)]['purposes_lemmatized'].count())

wedding_debt_0 = (data_purpose_dict
                  [(data_purpose_dict['purposes_lemmatized'] == 'свадьба')
                   &(data_purpose_dict['debt'] == 0)]['purposes_lemmatized'].count())
wedding_debt_1 = (data_purpose_dict
                  [(data_purpose_dict['purposes_lemmatized'] == 'свадьба')
                   &(data_purpose_dict['debt'] == 1)]['purposes_lemmatized'].count())

property_debt_0 = (data_purpose_dict
                   [(data_purpose_dict['purposes_lemmatized'] == 'недвижимость')
                    &(data_purpose_dict['debt'] == 0)]['purposes_lemmatized'].count())
property_debt_1 = (data_purpose_dict
                   [(data_purpose_dict['purposes_lemmatized'] == 'недвижимость')
                    &(data_purpose_dict['debt'] == 1)]['purposes_lemmatized'].count())

accommodation_debt_0 = (data_purpose_dict
                        [(data_purpose_dict['purposes_lemmatized'] == 'жилье')
                         &(data_purpose_dict['debt'] == 0)]['purposes_lemmatized'].count())
accommodation_debt_1 = (data_purpose_dict
                        [(data_purpose_dict['purposes_lemmatized'] == 'жилье')
                         &(data_purpose_dict['debt'] == 1)]['purposes_lemmatized'].count())

### Вывод

Для ответа на вопросы потребуется использование четырёх словарей:
- Для установления зависимости между наличием детей и возвратом кредита в срок удобнее всего разделить выборку на 2 большие группы. В первую группу следует определить тех клиентов, у которых есть дети, во вторую — тех, у кого детей нет;


- Словарь, в который включён столбец, содержащий семейное положение, не требует дополнений;


- Для выявления зависимости между уровнем дохода и возвратом кредита в срок необходимо разделить клиентов по уровню дохода, так как значений в столбце *'total_income'* слишком много. Было принято решение разбить выборку на 5 групп с применением двадцатого, сорокового, шестидесятого, восьмидесятого и сотого процентилей;


- Для исследования влияния разных целей кредита на его своевременный возврат достаточно лемматизированных значений, занесённых в столбец *'purpose_lemmatized'*.

### 3. Вопросы

**1. Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Узнаем, насколько отличаются доли должников среди клиентов, имеющих минимум одного ребёнка, и клиентов, не имеющих детей:

In [48]:
print('Доля должников среди клиентов, имеющих детей: {:.2%}'
      .format(children_1_debt_1/children_1_debt_0))
print('Доля должников среди клиентов, не имеющих детей: {:.2%}'
      .format(children_0_debt_1/children_0_debt_0))

Доля должников среди клиентов, имеющих детей: 10.14%
Доля должников среди клиентов, не имеющих детей: 8.16%


### Вывод

Доля должников выше среди клиентов, у которых есть дети.

**2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Выведем на экран процент должников в каждой из категорий:

In [49]:
print('Доля должников среди неженатых клиентов: {:.2%}'
      .format(single_debt_1/single_debt_0))
print('Доля должников среди клиентов, состоящих в гражданском браке: {:.2%}'
      .format(civil_marriage_debt_1/civil_marriage_debt_0))
print('Доля должников среди женатых клиентов: {:.2%}'
      .format(married_debt_1/married_debt_0))
print('Доля должников среди разведённых клиентов: {:.2%}'
      .format(divorced_debt_1/divorced_debt_0))
print('Доля должников среди овдовевших клиентов: {:.2%}'
      .format(widower_debt_1/widower_debt_0))

Доля должников среди неженатых клиентов: 10.83%
Доля должников среди клиентов, состоящих в гражданском браке: 10.31%
Доля должников среди женатых клиентов: 8.16%
Доля должников среди разведённых клиентов: 7.73%
Доля должников среди овдовевших клиентов: 6.95%


### Вывод

Вероятность возврата кредита в срок выше всего среди овдовевших и разведённых клиентов, ниже всего — среди неженатых и состоящих в гражданском браке.

**3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Вычислим процент должников для каждой из пяти групп:

In [50]:
print('Доля должников среди клиентов с низким уровнем дохода: {:.2%}'
      .format(percentile_20_debt_1/percentile_20_debt_0))
print('Доля должников среди клиентов с уровнем дохода ниже среднего: {:.2%}'
      .format(percentile_40_debt_1/percentile_40_debt_0))
print('Доля должников среди клиентов со средним уровнем дохода: {:.2%}'
      .format(percentile_60_debt_1/percentile_60_debt_0))
print('Доля должников среди клиентов с уровнем дохода выше среднего: {:.2%}'
      .format(percentile_80_debt_1/percentile_80_debt_0))
print('Доля должников среди клиентов с высоким уровнем дохода: {:.2%}'
      .format(percentile_100_debt_1/percentile_100_debt_0))

Доля должников среди клиентов с низким уровнем дохода: 8.73%
Доля должников среди клиентов с уровнем дохода ниже среднего: 8.98%
Доля должников среди клиентов со средним уровнем дохода: 9.49%
Доля должников среди клиентов с уровнем дохода выше среднего: 9.71%
Доля должников среди клиентов с высоким уровнем дохода: 8.08%


### Вывод

Обнаружена тенденция к снижению вероятности возврата кредита в срок с ростом уровня дохода. Однако клиенты с самым высоким уровнем дохода, напротив, возвращают кредит вовремя чаще.

**4. Как разные цели кредита влияют на его возврат в срок?**

In [51]:
print('Доля должников среди клиентов, которые взяли кредит для покупки автомобиля: {:.2%}'
      .format(car_debt_1/car_debt_0))
print('Доля должников среди клиентов, которые взяли кредит для получения образования: {:.2%}'
      .format(education_debt_1/education_debt_0))
print('Доля должников среди клиентов, которые взяли кредит для организации свадьбы: {:.2%}'
      .format(wedding_debt_1/wedding_debt_0 ))
print('Доля должников среди клиентов, которые взяли кредит для покупки недвижимости: {:.2%}'
      .format(property_debt_1/property_debt_0))
print('Доля должников среди клиентов, которые взяли кредит для покупки жилья: {:.2%}'
      .format(accommodation_debt_1/accommodation_debt_0))

Доля должников среди клиентов, которые взяли кредит для покупки автомобиля: 10.30%
Доля должников среди клиентов, которые взяли кредит для получения образования: 10.21%
Доля должников среди клиентов, которые взяли кредит для организации свадьбы: 8.65%
Доля должников среди клиентов, которые взяли кредит для покупки недвижимости: 8.08%
Доля должников среди клиентов, которые взяли кредит для покупки жилья: 7.41%


### Вывод

Те клиенты, которые брали кредиты для приобретения жилья и недвижимости, возвращали их в срок чаще. Среди клиентов, взявших кредит для покупки автомобиля и для получения образования, напротив, должников больше.

### 4. Общий вывод

Анализ данных выявил факторы, влияющие на вероятность своевременного возврата кредита.
Данная вероятность выше всего среди клиентов, у которых нет детей, нет супруги / супруга, уровень дохода низкий или ниже среднего. Приобретение жилья и недвижимости — цели кредита, также способствующие своевременному возврату.